In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2 as cv
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg19 import VGG19
#EfficientNetV2L
#from keras.applications.efficientnet import EfficientNetV2L
from keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.mobilenet import MobileNet
from sklearn.ensemble import VotingClassifier

In [ ]:
def input_data(folder_path, output_data):    #importing the data into the output_data list
    for dirs in os.listdir(folder_path):
        class_name = dirs
        new_path = os.path.join(folder_path, class_name)
        for img in os.listdir(new_path):
            img_arr = cv.imread(os.path.join(new_path, img), cv.IMREAD_COLOR)
            resize = cv.resize(img_arr, (128,128))
            img = cv.cvtColor(img_arr, cv.COLOR_RGB2BGR)
            output_data.append([resize, class_name])
    return output_data

In [ ]:
train_data = input_data(r"C:\Users\lenovo\Documents\dataset\melanoma_cancer_dataset\train", [])
test_data = input_data(r"C:\Users\lenovo\Documents\dataset\melanoma_cancer_dataset\test", [])

In [ ]:
np.random.shuffle(train_data)   #shuffling the data
np.random.shuffle(test_data)

In [ ]:
X_train = []              #separating the image and labels from the train_data list
Y_train = []
for features, labels in train_data:
    X_train.append(features)
    Y_train.append(labels)

In [ ]:
X_test = []    #separating the image and labels from the test_data list
Y_test = []
for features, labels in test_data:
    X_test.append(features)
    Y_test.append(labels)

In [ ]:
#Convert lists to arrays
test_images = np.array(X_test)
test_labels = np.array(Y_test)

In [ ]:
train_images = np.array(X_train)
train_labels = np.array(Y_train)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [ ]:
y_train[10:20]

In [ ]:
x_train.shape

In [ ]:
labels=np.unique(y_test)
print(labels)

# Vision Transformer Model0

In [ ]:
from .VisionTransformer import ViT
Model = ViT(input_shape = (128, 128, 3), num_classes = 2, num_transformer_blocks = 4,embed_dim = 32 , num_heads = 8 , ff_dim = 32)

In [ ]:
for layer in Model.layers:
    layer.trainable = False

print(Model.summary())

# Feature Extraction

In [ ]:
#Now, let us use features from convolutional network for RF
feature_extractor__=Model.predict(x_train)

In [ ]:
fe___test=Model.predict(x_test)
#fe___test=MobileNet_model.predict(x_test)

In [ ]:
features__ = feature_extractor__.reshape(feature_extractor__.shape[0], -1)

In [ ]:
f___test=fe___test.reshape(fe___test.shape[0],-1)

In [ ]:
print(f___test.shape)

In [ ]:
# X_for_RF = features__ #This is our X input to RF

#Machine Learning Models For Classifications

##K-Nearest Neighbours

In [ ]:
import math
import numpy as np
import pandas as pd
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline
# plt.style.use('seaborn-whitegrid')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(features__ , y_train)
y_pred=knn.predict(f___test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix , precision_score,   recall_score ,  f1_score
acc=accuracy_score(y_test,y_pred)
acc

In [ ]:
precision_score(y_test,y_pred,average='micro')

In [ ]:
recall_score(y_test,y_pred,average='micro')

In [ ]:
f1_score(y_test,y_pred,average='micro')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_KNN.png')
plt.show()

##Logistic Regression

###Used In  Binary and Multiclass Classifications

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
features__.shape

In [ ]:
...
# define the multinomial logistic regression model
lrm = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lrm.fit(features__ , y_train)
y_pred_lr=lrm.predict(f___test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix , precision_score,   recall_score ,  f1_score
acc=accuracy_score(y_test,y_pred_lr)
acc

In [ ]:
precision_score(y_test,y_pred_lr,average=None)

In [ ]:
recall_score(y_test,y_pred_lr,average=None)

In [ ]:
f1_score(y_test,y_pred_lr,average=None)

In [ ]:
cm = confusion_matrix(y_test,y_pred_lr)
cm

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/XceptionLR.png')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score,   recall_score ,  f1_score
acc=accuracy_score(y_test,y_pred_lr)
acc

In [ ]:
# precision_score(y_test,y_pred,average=None)

In [ ]:
# metrics.f1_score(y_test,y_pred,average=None)

##Random Forest

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
# Train the model on training data
RF_model.fit(features__, y_train) #For sklearn no one hot encoding

In [ ]:
print(RF_model.score(features__, y_train))

In [ ]:
#Send test data through same feature extractor process
X_test_feature = Model.predict(x_test)



In [ ]:
X_test_features__ = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
print(X_test_feature.shape)

In [ ]:
#Now predict using the trained RF model.
prediction_RF = RF_model.predict(X_test_features__)
#Inverse le transform to get original label back.
prediction_RF = le.inverse_transform(prediction_RF)

In [ ]:
print(prediction_RF.shape)

In [ ]:
print(test_labels.shape)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF))

In [ ]:
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF))
print("Precision =", precision_score(test_labels,prediction_RF,average='micro'))
print("Recall =",recall_score(test_labels,prediction_RF,average='micro'))
print("F_1 =", f1_score(test_labels,prediction_RF,average='micro'))

In [ ]:
cm = confusion_matrix(test_labels,prediction_RF)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_RandomForest.png')
plt.show()

##XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
xgb_classifier = xgb.XGBClassifier(tree_method = "auto")

In [ ]:
xgb_classifier.fit(features__, y_train)

In [ ]:
predictions = xgb_classifier.predict(X_test_features__)
prediction_XG = le.inverse_transform(predictions)

In [ ]:
print("Accuracy of Model::",metrics.accuracy_score(test_labels, prediction_XG))

In [ ]:
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_XG))
print("Precision =", precision_score(test_labels, prediction_XG,average='micro'))
print("Recall =",recall_score(test_labels, prediction_XG,average='micro'))
print("F_1 =", f1_score(test_labels, prediction_XG,average='micro'))

In [ ]:
cm = confusion_matrix(test_labels, prediction_XG)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_XGBoost.png')
plt.show()

##SVC- Support Vector Classifier

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC(C=0.65, random_state=0, kernel='rbf')

In [ ]:
svc.fit(features__, y_train)

In [ ]:
X_test_features__ = X_test_feature.reshape(X_test_feature.shape[0], -1)
X_test_features__.shape

In [ ]:
X_test_features = Model.predict(x_test)
X_test_features.shape

In [ ]:
predictions = svc.predict(X_test_features__ )
prediction_SVC = le.inverse_transform(predictions)

In [ ]:
prediction_SVC

In [ ]:
print("Accuracy of Model::",metrics.accuracy_score(test_labels, prediction_SVC))

In [ ]:
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_SVC))
print("Precision =", precision_score(test_labels, prediction_SVC,average='micro'))
print("Recall =",recall_score(test_labels, prediction_SVC,average='micro'))
print("F_1 =", f1_score(test_labels, prediction_SVC,average='micro'))

In [ ]:
cm = confusion_matrix(test_labels, prediction_SVC)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_SVC.png')
plt.show()

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
#>>> X = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
X_embedded = TSNE(n_components=3, learning_rate='auto', init='random', perplexity=3).fit_transform(features__)
X_embedded.shape

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()
df["y"] = y_train
df["comp-1"] = X_embedded[:,0]
df["comp-2"] = X_embedded[:,1]

In [ ]:
sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                data=df).set(title="")
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
# plt.savefig('/home/nd/Ankit-Deepto-Btech/__128_128/TSNE_Train_data.png')
plt.show()

##ENSEMBLE

In [ ]:
final_model = VotingClassifier(estimators=[('rf', RF_model), ('xgb', xgb_classifier),('knn',knn),('svc',svc),('lr',lrm)], voting='hard')


In [ ]:
final_model.fit(features__, y_train)

In [ ]:
predictions = final_model.predict(X_test_features__) # X_test_features
prediction_final = le.inverse_transform(predictions)

In [ ]:
print("Accuracy of Model::",metrics.accuracy_score(test_labels, prediction_final))

In [ ]:
print("Precision =", precision_score(test_labels, prediction_final,average='micro'))
print("Recall =",recall_score(test_labels, prediction_final,average='micro'))
print("F_1 =", f1_score(test_labels, prediction_final,average='micro'))

In [ ]:
print("Accuracy =",accuracy_score(test_labels, prediction_final))
print("Precision =", precision_score(test_labels, prediction_final,average=None))
print("Recall =",recall_score(test_labels, prediction_final,average=None))
print("F_1 =", f1_score(test_labels, prediction_final,average=None))

In [ ]:
cm = confusion_matrix(test_labels, prediction_final)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_Ensemble.png')
plt.show()

# CapsNet Model1

In [ ]:
from .Capsnet import CapsNet
Model = CapsNet(input_shape=(128, 128, 3) , num_classes=3)

In [ ]:
#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in Model.layers:
    layer.trainable = False

Model.summary()  #Trainable parameters will be 0

In [ ]:
#Now, let us use features from convolutional network for RF
feature_extractor__=Model.predict(x_train)

In [ ]:
fe___test=Model.predict(x_test)
#fe___test=MobileNet_model.predict(x_test)

In [ ]:
features__ = feature_extractor__.reshape(feature_extractor__.shape[0], -1)

In [ ]:
f___test=fe___test.reshape(fe___test.shape[0],-1)

In [ ]:
print(f___test.shape)

In [ ]:
# X_for_RF = features__ #This is our X input to RF

#Machine Learning Models For Classifications

##K-Nearest Neighbours

In [ ]:
import math
import numpy as np
import pandas as pd
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline
# plt.style.use('seaborn-whitegrid')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(features__ , y_train)
y_pred=knn.predict(f___test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix , precision_score,   recall_score ,  f1_score
acc=accuracy_score(y_test,y_pred)
acc

In [ ]:
precision_score(y_test,y_pred,average='micro')

In [ ]:
recall_score(y_test,y_pred,average='micro')

In [ ]:
f1_score(y_test,y_pred,average='micro')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_KNN.png')
plt.show()

##Logistic Regression

###Used In  Binary and Multiclass Classifications

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
features__.shape

In [ ]:
...
# define the multinomial logistic regression model
lrm = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lrm.fit(features__ , y_train)
y_pred_lr=lrm.predict(f___test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix , precision_score,   recall_score ,  f1_score
acc=accuracy_score(y_test,y_pred_lr)
acc

In [ ]:
precision_score(y_test,y_pred_lr,average=None)

In [ ]:
recall_score(y_test,y_pred_lr,average=None)

In [ ]:
f1_score(y_test,y_pred_lr,average=None)

In [ ]:
cm = confusion_matrix(y_test,y_pred_lr)
cm

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/XceptionLR.png')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix , precision_score,   recall_score ,  f1_score
acc=accuracy_score(y_test,y_pred_lr)
acc

##Random Forest

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
# Train the model on training data
RF_model.fit(features__, y_train) #For sklearn no one hot encoding

In [ ]:
print(RF_model.score(features__, y_train))

In [ ]:
#Send test data through same feature extractor process
X_test_feature = Model.predict(x_test)



In [ ]:
X_test_features__ = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
print(X_test_feature.shape)

In [ ]:
#Now predict using the trained RF model.
prediction_RF = RF_model.predict(X_test_features__)
#Inverse le transform to get original label back.
prediction_RF = le.inverse_transform(prediction_RF)

In [ ]:
print(prediction_RF.shape)

In [ ]:
print(test_labels.shape)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF))

In [ ]:
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF))
print("Precision =", precision_score(test_labels,prediction_RF,average='micro'))
print("Recall =",recall_score(test_labels,prediction_RF,average='micro'))
print("F_1 =", f1_score(test_labels,prediction_RF,average='micro'))

In [ ]:
cm = confusion_matrix(test_labels,prediction_RF)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_RandomForest.png')
plt.show()

##XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
xgb_classifier = xgb.XGBClassifier(tree_method = "auto")

In [ ]:
xgb_classifier.fit(features__, y_train)

In [ ]:
predictions = xgb_classifier.predict(X_test_features__)
prediction_XG = le.inverse_transform(predictions)

In [ ]:
print("Accuracy of Model::",metrics.accuracy_score(test_labels, prediction_XG))

In [ ]:
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_XG))
print("Precision =", precision_score(test_labels, prediction_XG,average='micro'))
print("Recall =",recall_score(test_labels, prediction_XG,average='micro'))
print("F_1 =", f1_score(test_labels, prediction_XG,average='micro'))

In [ ]:
cm = confusion_matrix(test_labels, prediction_XG)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_XGBoost.png')
plt.show()

##SVC- Support Vector Classifier

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC(C=0.65, random_state=0, kernel='rbf')

In [ ]:
svc.fit(features__, y_train)

In [ ]:
X_test_features__ = X_test_feature.reshape(X_test_feature.shape[0], -1)
X_test_features__.shape

In [ ]:
X_test_features = Model.predict(x_test)
X_test_features.shape

In [ ]:
predictions = svc.predict(X_test_features__ )
prediction_SVC = le.inverse_transform(predictions)

In [ ]:
prediction_SVC

In [ ]:
print("Accuracy of Model::",metrics.accuracy_score(test_labels, prediction_SVC))

In [ ]:
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_SVC))
print("Precision =", precision_score(test_labels, prediction_SVC,average='micro'))
print("Recall =",recall_score(test_labels, prediction_SVC,average='micro'))
print("F_1 =", f1_score(test_labels, prediction_SVC,average='micro'))

In [ ]:
cm = confusion_matrix(test_labels, prediction_SVC)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_SVC.png')
plt.show()

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
#>>> X = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
X_embedded = TSNE(n_components=3, learning_rate='auto', init='random', perplexity=3).fit_transform(features__)
X_embedded.shape

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()
df["y"] = y_train
df["comp-1"] = X_embedded[:,0]
df["comp-2"] = X_embedded[:,1]

In [ ]:
sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                data=df).set(title="")
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
# plt.savefig('/home/nd/Ankit-Deepto-Btech/__128_128/TSNE_Train_data.png')
plt.show()

##ENSEMBLE

In [ ]:
final_model_1 = VotingClassifier(estimators=[('rf', RF_model), ('xgb', xgb_classifier),('knn',knn),('svc',svc),('lr',lrm)], voting='hard')


In [ ]:
final_model_1.fit(features__, y_train)

In [ ]:
predictions = final_model_1.predict(X_test_features__) # X_test_features
prediction_final = le.inverse_transform(predictions)

In [ ]:
print("Accuracy of Model::",metrics.accuracy_score(test_labels, prediction_final))

In [ ]:
print("Precision =", precision_score(test_labels, prediction_final,average='micro'))
print("Recall =",recall_score(test_labels, prediction_final,average='micro'))
print("F_1 =", f1_score(test_labels, prediction_final,average='micro'))

In [ ]:
print("Accuracy =",accuracy_score(test_labels, prediction_final))
print("Precision =", precision_score(test_labels, prediction_final,average=None))
print("Recall =",recall_score(test_labels, prediction_final,average=None))
print("F_1 =", f1_score(test_labels, prediction_final,average=None))

In [ ]:
cm = confusion_matrix(test_labels, prediction_final)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_Ensemble.png')
plt.show()

# VGG19  Model2

In [ ]:
Model = VGG19(weights='imagenet', include_top=False, input_shape=(128,128, 3))

In [ ]:
#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in Model.layers:
	layer.trainable = False

Model.summary()  #Trainable parameters will be 0

In [ ]:
#Now, let us use features from convolutional network for RF
feature_extractor__=Model.predict(x_train)

In [ ]:
fe___test=Model.predict(x_test)
#fe___test=MobileNet_model.predict(x_test)

In [ ]:
features__ = feature_extractor__.reshape(feature_extractor__.shape[0], -1)

In [ ]:
f___test=fe___test.reshape(fe___test.shape[0],-1)

In [ ]:
print(f___test.shape)

In [ ]:
# X_for_RF = features__ #This is our X input to RF

#Machine Learning Models For Classifications

##K-Nearest Neighbours

In [ ]:
import math
import numpy as np
import pandas as pd
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline
# plt.style.use('seaborn-whitegrid')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(features__ , y_train)
y_pred=knn.predict(f___test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix , precision_score,   recall_score ,  f1_score
acc=accuracy_score(y_test,y_pred)
acc

In [ ]:
precision_score(y_test,y_pred,average='micro')

In [ ]:
recall_score(y_test,y_pred,average='micro')

In [ ]:
f1_score(y_test,y_pred,average='micro')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_KNN.png')
plt.show()

##Logistic Regression

###Used In  Binary and Multiclass Classifications

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
features__.shape

In [ ]:
...
# define the multinomial logistic regression model
lrm = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lrm.fit(features__ , y_train)
y_pred_lr=lrm.predict(f___test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix , precision_score,   recall_score ,  f1_score
acc=accuracy_score(y_test,y_pred_lr)
acc

In [ ]:
precision_score(y_test,y_pred_lr,average=None)

In [ ]:
recall_score(y_test,y_pred_lr,average=None)

In [ ]:
f1_score(y_test,y_pred_lr,average=None)

In [ ]:
cm = confusion_matrix(y_test,y_pred_lr)
cm

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/XceptionLR.png')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix , precision_score,   recall_score ,  f1_score
acc=accuracy_score(y_test,y_pred_lr)
acc

##Random Forest

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
# Train the model on training data
RF_model.fit(features__, y_train) #For sklearn no one hot encoding

In [ ]:
print(RF_model.score(features__, y_train))

In [ ]:
#Send test data through same feature extractor process
X_test_feature = Model.predict(x_test)



In [ ]:
X_test_features__ = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
print(X_test_feature.shape)

In [ ]:
#Now predict using the trained RF model.
prediction_RF = RF_model.predict(X_test_features__)
#Inverse le transform to get original label back.
prediction_RF = le.inverse_transform(prediction_RF)

In [ ]:
print(prediction_RF.shape)

In [ ]:
print(test_labels.shape)

In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF))

In [ ]:
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF))
print("Precision =", precision_score(test_labels,prediction_RF,average='micro'))
print("Recall =",recall_score(test_labels,prediction_RF,average='micro'))
print("F_1 =", f1_score(test_labels,prediction_RF,average='micro'))

In [ ]:
cm = confusion_matrix(test_labels,prediction_RF)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_RandomForest.png')
plt.show()

##XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
xgb_classifier = xgb.XGBClassifier(tree_method = "auto")

In [ ]:
xgb_classifier.fit(features__, y_train)

In [ ]:
predictions = xgb_classifier.predict(X_test_features__)
prediction_XG = le.inverse_transform(predictions)

In [ ]:
print("Accuracy of Model::",metrics.accuracy_score(test_labels, prediction_XG))

In [ ]:
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_XG))
print("Precision =", precision_score(test_labels, prediction_XG,average='micro'))
print("Recall =",recall_score(test_labels, prediction_XG,average='micro'))
print("F_1 =", f1_score(test_labels, prediction_XG,average='micro'))

In [ ]:
cm = confusion_matrix(test_labels, prediction_XG)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_XGBoost.png')
plt.show()

##SVC- Support Vector Classifier

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC(C=0.65, random_state=0, kernel='rbf')

In [ ]:
svc.fit(features__, y_train)

In [ ]:
X_test_features__ = X_test_feature.reshape(X_test_feature.shape[0], -1)
X_test_features__.shape

In [ ]:
X_test_features = Model.predict(x_test)
X_test_features.shape

In [ ]:
predictions = svc.predict(X_test_features__ )
prediction_SVC = le.inverse_transform(predictions)

In [ ]:
prediction_SVC

In [ ]:
print("Accuracy of Model::",metrics.accuracy_score(test_labels, prediction_SVC))

In [ ]:
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_SVC))
print("Precision =", precision_score(test_labels, prediction_SVC,average='micro'))
print("Recall =",recall_score(test_labels, prediction_SVC,average='micro'))
print("F_1 =", f1_score(test_labels, prediction_SVC,average='micro'))

In [ ]:
cm = confusion_matrix(test_labels, prediction_SVC)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_SVC.png')
plt.show()

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
#>>> X = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
X_embedded = TSNE(n_components=3, learning_rate='auto', init='random', perplexity=3).fit_transform(features__)
X_embedded.shape

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()
df["y"] = y_train
df["comp-1"] = X_embedded[:,0]
df["comp-2"] = X_embedded[:,1]

In [ ]:
sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                data=df).set(title="")
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
# plt.savefig('/home/nd/Ankit-Deepto-Btech/__128_128/TSNE_Train_data.png')
plt.show()

##ENSEMBLE

In [ ]:
final_model_2 = VotingClassifier(estimators=[('rf', RF_model), ('xgb', xgb_classifier),('knn',knn),('svc',svc),('lr',lrm)], voting='hard')


In [ ]:
final_model_2.fit(features__, y_train)

In [ ]:
predictions = final_model_2.predict(X_test_features__) # X_test_features
prediction_final = le.inverse_transform(predictions)

In [ ]:
print("Accuracy of Model::",metrics.accuracy_score(test_labels, prediction_final))

In [ ]:
print("Precision =", precision_score(test_labels, prediction_final,average='micro'))
print("Recall =",recall_score(test_labels, prediction_final,average='micro'))
print("F_1 =", f1_score(test_labels, prediction_final,average='micro'))

In [ ]:
print("Accuracy =",accuracy_score(test_labels, prediction_final))
print("Precision =", precision_score(test_labels, prediction_final,average=None))
print("Recall =",recall_score(test_labels, prediction_final,average=None))
print("F_1 =", f1_score(test_labels, prediction_final,average=None))

In [ ]:
cm = confusion_matrix(test_labels, prediction_final)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/Xception_128_128/Xception_Ensemble.png')
plt.show()

# Ensemble of ensemble code

In [ ]:
final_model_final= VotingClassifier(estimators=[('model0', final_model), ('model1', final_model_1),('model2',final_model_2)], voting='hard')


In [ ]:
final_model_final.fit(features__, y_train)

In [ ]:
predictions = final_model_final.predict(X_test_features__) # X_test_features
prediction_final = le.inverse_transform(predictions)

In [ ]:
print("Accuracy of Model::",metrics.accuracy_score(test_labels, prediction_final))

In [ ]:
print("Precision =", precision_score(test_labels, prediction_final,average='micro'))
print("Recall =",recall_score(test_labels, prediction_final,average='micro'))
print("F_1 =", f1_score(test_labels, prediction_final,average='micro'))

In [ ]:
print("Accuracy =",accuracy_score(test_labels, prediction_final))
print("Precision =", precision_score(test_labels, prediction_final,average=None))
print("Recall =",recall_score(test_labels, prediction_final,average=None))
print("F_1 =", f1_score(test_labels, prediction_final,average=None))

In [ ]:
cm = confusion_matrix(test_labels, prediction_final)
import seaborn as sns
import matplotlib.pyplot as plt

### Confusion Matrix
from sklearn.metrics import confusion_matrix
#predictions = model.predict(x_test, steps=len(x_test), verbose=0)
#y_pred=model.predict(x_test)
#y_pred = np.round(y_pred)
#y_pred = np.argmax(predictions, axis=-1)

#y_true=np.argmax(y_test, axis=-1)

#cm = confusion_matrix(y_true, y_pred)

## Get Class Labels
#labels = le.classes_
class_names = labels

# Plot confusion matrix in a beautiful manner
fig = plt.figure(figsize=(7, 7))
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cmap="Blues", fmt = "g"); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted', fontsize=15)
ax.xaxis.set_label_position('bottom')
plt.xticks(rotation=90)
ax.xaxis.set_ticklabels(class_names, fontsize = 15)
ax.xaxis.tick_bottom()

ax.set_ylabel('True', fontsize=15)
ax.yaxis.set_ticklabels(class_names, fontsize = 15)
plt.yticks(rotation=0)

#plt.title('Refined Confusion Matrix', fontsize=15)

# plt.savefig('/home/nd/Ankit-Deepto-Btech/ensemble_of_ensemble/confusion_matrix.png')
plt.show()